# Support Vector Machines


Imports


In [1]:
from sklearn.svm import SVC, SVR
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
    r2_score,
)
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from binning import bin

Read the train and test datasets


In [3]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Convert training set OHE columns to int8


In [4]:
columns_to_convert = train_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
train_df[columns_to_convert] = train_df[columns_to_convert].astype("int8")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385516 entries, 0 to 385515
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 1.5 GB


Convert testing set OHE columns to int8


In [5]:
columns_to_convert = test_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
test_df[columns_to_convert] = test_df[columns_to_convert].astype("int8")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96379 entries, 0 to 96378
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 385.9 MB


Get the training features and label


In [4]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [5]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [6]:
kf = KFold(n_splits=5)

## SVM Classifier


Bin the label


In [5]:
y_train_binned = bin(y_train)
y_test_binned = bin(y_test)

Initialize an SVM classifier


In [7]:
svc = SVC(kernel="sigmoid")

Initialize scores lists


In [8]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

Train the model using 5-fold cross-validation


In [9]:
fold = 1
for train, test in kf.split(X_train, y_train_binned):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    svc.fit(X_train[train], y_train_binned[train])

    # Predict on the test set
    predictions = svc.predict(X_train[test])

    # Evaluate the model
    precision = precision_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train_binned[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.3431420358765928
Recall = 0.4549828802656153
Accuracy = 0.4549828802656153
F1 score = 0.3685845663349058

##### FOLD: 2 #####
Precision = 0.3845940816507721
Recall = 0.4535750878694733
Accuracy = 0.4535750878694733
F1 score = 0.3652991003307992

##### FOLD: 3 #####
Precision = 0.3453755812018709
Recall = 0.45363993618925336
Accuracy = 0.45363993618925336
F1 score = 0.36352206954341204

##### FOLD: 4 #####
Precision = 0.3438909715047536
Recall = 0.45397714745210954
Accuracy = 0.45397714745210954
F1 score = 0.3682595240535108

##### FOLD: 5 #####
Precision = 0.34421652279291604
Recall = 0.45295254399958496
Accuracy = 0.45295254399958496
F1 score = 0.366438883973592



Print the mean scores of the folds


In [10]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.35224383860538105
Mean Recall = 0.4538255191552073
Mean Accuracy = 0.4538255191552073
Mean F1 score = 0.366420828847244


Make predictions on the test set


In [11]:
predictions = svc.predict(X_test)

Evaluate the model


In [12]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     11718
           1       0.46      0.85      0.60     42216
           2       0.41      0.24      0.31     32447
           3       0.00      0.00      0.00      9998

    accuracy                           0.45     96379
   macro avg       0.22      0.27      0.23     96379
weighted avg       0.34      0.45      0.37     96379



## SVM Regressor


Initialize a RandomForestRegressor


In [7]:
svr = SVR(kernel="sigmoid")

Initialize scores lists


In [8]:
r2_scores = []

Train the model using 5-fold cross-validation


In [9]:
fold = 1
for train, test in kf.split(X_train, y_train):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    svr.fit(X_train[train], y_train[train])

    # Predict on the test set
    predictions = svr.predict(X_train[test])

    # Evaluate the model
    r2 = r2_score(y_true=y_train[test], y_pred=predictions)

    # Store the result
    r2_scores.append(r2)

    # Print the scores for each fold
    print(f"R2 score = {r2}")

    fold += 1

##### FOLD: 1 #####
R2 score = -0.016008226566073258
##### FOLD: 2 #####
R2 score = -0.012595648888917621
##### FOLD: 3 #####
R2 score = -0.013120615720138007
##### FOLD: 4 #####
R2 score = -0.01634761823539299
##### FOLD: 5 #####
R2 score = -0.013100746674741481


Print the mean scores of the folds


In [10]:
print(f"Mean R2 score = {np.mean(r2_scores)}")

Mean R2 score = -0.014234571217052672


Make predictions on the test set


In [11]:
predictions = svr.predict(X_test)

Evaluate the model


In [12]:
print(r2_score(y_true=y_test, y_pred=predictions))

-0.01595606236714242
